# Data

1. Gets necessary data from CRSP for S&P constituents
2. Transforms PDFs of FOMC statements to URLs (so that all FOMCs are now in a url)
3. Scrapes necessary text from all URLs
4. Removes noise from FOMC statements

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import wrds

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nathanueda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Step 1: Get S&P Data

In [4]:
conn=wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [101]:
# get daily returns
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.dsp500list as a,
                        crsp.dsf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        and b.date<='12/31/2023'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [102]:
# add other descriptive variables
mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]

In [103]:
# linking with compustat through ccm
ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]
ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip', 'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]

In [104]:
# Retrieve GICS codes from Compustat company table
comp_gics = conn.raw_sql("""
    SELECT gvkey, gsector, ggroup, gind
    FROM comp.company
    """)


In [105]:
# Merge GICS codes into sp500ccm
sp500ccm = pd.merge(sp500ccm, comp_gics, on='gvkey', how='left')


In [106]:
# drop unnec cols
sp500 = sp500ccm.drop(columns=['shrcd', 'exchcd', 'hsiccd', 'iid', 'start', 'ending', 'ncusip'])

In [107]:
sp500['gsector'] = sp500['gsector'].astype(int)
sp500['gind'] = sp500['gind'].astype(int)
sp500['ggroup'] = sp500['ggroup'].astype(int)

In [118]:
# Map GICS codes to sector names
gics_sector_map = {
    10: 'Energy',
    15: 'Materials',
    20: 'Industrials',
    25: 'Consumer Discretionary',
    30: 'Consumer Staples',
    35: 'Health Care',
    40: 'Financials',
    45: 'Information Technology',
    50: 'Communication Services',
    55: 'Utilities',
    60: 'Real Estate'
}

gics_industry_group_map = {
    1010: 'Energy',
    1510: 'Materials',
    2010: 'Capital Goods',
    2020: 'Commercial & Professional Services',
    2030: 'Transportation',
    2510: 'Automobiles & Components',
    2520: 'Consumer Durables & Apparel',
    2530: 'Consumer Services',
    2540: 'Retailing',
    2550: 'Media & Entertainment',
    3010: 'Food & Staples Retailing',
    3020: 'Food, Beverage & Tobacco',
    3030: 'Household & Personal Products',
    3510: 'Health Care Equipment & Services',
    3520: 'Pharmaceuticals, Biotechnology & Life Sciences',
    4010: 'Banks',
    4020: 'Diversified Financials',
    4030: 'Insurance',
    4040: 'Real Estate',
    4510: 'Software & Services',
    4520: 'Technology Hardware & Equipment',
    4530: 'Semiconductors & Semiconductor Equipment',
    5010: 'Telecommunication Services',
    5020: 'Media & Entertainment',
    5510: 'Utilities',
    6010: 'Real Estate',
    6020: 'Real Estate Management & Development'
}


gics_industry_map = {
    101010: 'Oil, Gas & Consumable Fuels',
    101020: 'Energy Equipment & Services',
    151010: 'Chemicals',
    151020: 'Construction Materials',
    151030: 'Containers & Packaging',
    151040: 'Metals & Mining',
    151050: 'Paper & Forest Products',
    201010: 'Aerospace & Defense',
    201020: 'Building Products',
    201030: 'Construction & Engineering',
    201040: 'Electrical Equipment',
    201050: 'Industrial Conglomerates',
    201060: 'Machinery',
    201070: 'Marine',
    202010: 'Commercial Services & Supplies',
    202020: 'Professional Services',
    203010: 'Air Freight & Logistics',
    203020: 'Commercial Services & Supplies',
    203040: 'Passenger Airlines',
    251010: 'Auto Components',
    251020: 'Automobiles',
    252010: 'Consumer Durables',
    252020: 'Household Durables',
    252030: 'Leisure Products',
    253010: 'Hotels, Restaurants & Leisure',
    253020: 'Diversified Consumer Services',
    254010: 'Specialty Retail',
    255010: 'Media',
    255020: 'Entertainment',
    255030: 'Interactive Media & Services',
    255040: 'Publishing',
    301010: 'Food & Staples Retailing',
    302010: 'Beverages',
    302020: 'Food Products',
    302030: 'Tobacco',
    303010: 'Household Products',
    303020: 'Personal Products',
    351010: 'Health Care Equipment & Supplies',
    351020: 'Health Care Providers & Services',
    351030: 'Health Care Technology',
    352010: 'Pharmaceuticals',
    352020: 'Biotechnology',
    352030: 'Life Sciences Tools & Services',
    401010: 'Banks',
    401020: 'Diversified Banks',
    402010: 'Thrifts & Mortgage Finance',
    402020: 'Diversified Financial Services',
    402030: 'Consumer Finance',
    403010: 'Insurance',
    404020: 'Real Estate Management & Development',
    451010: 'Software',
    451020: 'IT Services',
    451030: 'Technology Hardware, Storage & Peripherals',
    452010: 'Semiconductors & Semiconductor Equipment',
    452020: 'Electronic Components',
    452030: 'Communications Equipment',
    453010: 'Electronic Equipment, Instruments & Components',
    501010: 'Diversified Telecommunication Services',
    501020: 'Wireless Telecommunication Services',
    502010: 'Media',
    502020: 'Broadcasting',
    502030: 'Cable & Satellite',
    551010: 'Electric Utilities',
    551020: 'Gas Utilities',
    551030: 'Multi-Utilities',
    551040: 'Water Utilities',
    551050: 'Independent Power Producers & Energy Traders',
    601010: 'Equity Real Estate Investment Trusts (REITs)',
    601025: 'Retail REITs',
    601030: 'Residential REITs',
    601040: 'Industrial REITs',
    601050: 'Office REITs',
    601060: 'Hotel & Resort REITs',
    601070: 'Health Care REITs',
    601080: 'Diversified REITs',
    602010: 'Real Estate Management & Development'
}


sp500['sector'] = sp500['gsector'].map(gics_sector_map)
sp500['industry'] = sp500['gind'].map(gics_industry_map)
sp500['group'] = sp500['ggroup'].map(gics_industry_group_map)

In [119]:
sp500

,date,permno,comnam,ticker,gvkey,ret,gsector,ggroup,gind,sector,industry,group
0,2000-01-03,64936,DOMINION RESOURCES INC VA,D,004029,-0.028662,55,5510,551030,Utilities,Multi-Utilities,Utilities
1,2000-01-03,24205,F P L GROUP INC,FPL,004517,-0.036496,55,5510,551010,Utilities,Electric Utilities,Utilities
2,2000-01-03,60441,MIRAGE RESORTS INC,MIR,005211,-0.028926,25,2530,253010,Consumer Discretionary,"Hotels, Restaurants & Leisure",Consumer Services
3,2000-01-03,45751,MARSH & MCLENNAN COS INC,MMC,007065,-0.011757,40,4030,403010,Financials,Insurance,Insurance
4,2000-01-03,76887,ALLIED WASTE INDUSTRIES INC,AW,022140,-0.042553,20,2020,202010,Industrials,Commercial Services & Supplies,Commercial & Professional Services
...,...,...,...,...,...,...,...,...,...,...,...,...
3021419,2023-12-29,87445,TELEDYNE TECHNOLOGIES,TDY,126721,0.004683,45,4520,452030,Information Technology,Communications Equipment,Technology Hardware & Equipment
3021420,2023-12-29,21792,CENTERPOINT ENERGY INC,CNP,005742,0.000350,55,5510,551030,Utilities,Multi-Utilities,Utilities
3021421,2023-12-29,13356,PHILLIPS 66,PSX,170841,0.002258,10,1010,101020,Energy,Energy Equipment & Services,Energy
3021422,2023-12-29,58819,ALLIANT ENERGY CORP,LNT,011554,-0.000390,55,5510,551010,Utilities,Electric Utilities,Utilities


In [120]:
sp500.to_pickle('data/sp500_constituents.pkl')

In [34]:
sp500.to_pickle('data/sp500.pkl')

In [35]:
sp500.isna().sum()

permno      0
date        0
ret       101
dtype: int64

In [36]:
(sp500[sp500['ret'].isna()])['permno'].nunique()

98

### Step 2: Scrape Relevant Text from URLs

In [3]:
# fomc_urls = pd.read_csv('data/fomc statement times pre 2011 with urls.csv')

# has original urls and urls from pdfs
fomc_urls = pd.read_csv('data/all_fomc_urls.csv')
# fomc_urls = fomc_urls.drop(columns=['Statement Time', 'Press Conference'])
fomc_urls = fomc_urls.rename(columns={'Statement Date': 'date', 'URL':'url'})
fomc_urls['date'] = pd.to_datetime(fomc_urls['date'])
fomc_urls

,date,url
0,1999-05-18,https://www.federalreserve.gov/boarddocs/press...
1,1999-06-30,https://www.federalreserve.gov/boarddocs/press...
2,1999-08-24,https://www.federalreserve.gov/boarddocs/press...
3,1999-10-05,https://www.federalreserve.gov/boarddocs/press...
4,1999-11-16,https://www.federalreserve.gov/boarddocs/press...
...,...,...
162,2023-09-20,https://www.federalreserve.gov/newsevents/pres...
163,2023-11-01,https://www.federalreserve.gov/newsevents/pres...
164,2023-12-13,https://www.federalreserve.gov/newsevents/pres...
165,2024-01-31,https://www.federalreserve.gov/newsevents/pres...


In [14]:
# get statement texts
fomc_statements = []

for index, row in fomc_urls.iterrows():
    date = row['date']
    url = row['url']
    
    response = requests.get(url)
    
    # request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        statement_text = soup.get_text(separator='\n', strip=True)

        content_div = soup.find('div', class_='col-xs-12 col-sm-8 col-md-8')

        # dates in range: 01-31-2006 - 01-26-2011 enter here and store relevant text in col-xs-12 col-sm-8 col-md-8
        # converted pdfs to urls as well which follow this format
        if content_div:
            paragraphs = content_div.find_all('p')
            statement_text = '\n'.join([p.get_text(strip=True) for p in paragraphs])
        
        # dates in range: 05-18-1999 - 12-13-2005 enter here and store relevant text in <p>
        else:
            statement_text= soup.find('p').get_text(separator='\n', strip=True)
        # add statment
        fomc_statements.append({'date': date, 'statement': statement_text})
    else:
        print(f"Failed to retrieve URL at index {index}: {url}")

In [71]:
fomc_statements = pd.DataFrame(fomc_statements)

print("Statements Format 1 (05-18-1999 - 12-13-2005):")
display(fomc_statements['statement'].iloc[0])

print("Statements Format 2 (01-31-2006 - 01-26-2011) and onwards:")

display(fomc_statements['statement'].iloc[-1])

Statements Format 1 (05-18-1999 - 12-13-2005):


"For immediate release\nThe Federal Reserve released the following statement after today's Federal Open Market Committee meeting:\nWhile the FOMC did not take action today to alter the stance of monetary policy, the Committee was concerned about the potential for a buildup of inflationary imbalances that could undermine the favorable performance of the economy and therefore adopted a directive that is tilted toward the possibility of a  firming in the stance of monetary policy.  Trend increases in costs and core prices have generally remained quite subdued.  But domestic financial markets have recovered and foreign economic prospects have improved since the easing of monetary policy last fall.  Against the background of already-tight domestic labor markets and ongoing strength in demand in excess of productivity gains, the Committee recognizes the need to be alert to developments over coming months that might indicate that financial conditions may no longer be consistent with containin

Statements Format 2 (01-31-2006 - 01-26-2011) and onwards:


"Recent indicators suggest that economic activity has been expanding at a solid pace. Job gains have remained strong, and the unemployment rate has remained low. Inflation has eased over the past year but remains elevated.\nThe Committee seeks to achieve maximum employment and inflation at the rate of 2 percent over the longer run. The Committee judges that the risks to achieving its employment and inflation goals are moving into better balance. The economic outlook is uncertain, and the Committee remains highly attentive to inflation risks.\nIn support of its goals, the Committee decided to maintain the target range for the federal funds rate at 5-1/4 to 5-1/2 percent. In considering any adjustments to the target range for the federal funds rate, the Committee will carefully assess incoming data, the evolving outlook, and the balance of risks. The Committee does not expect it will be appropriate to reduce the target range until it has gained greater confidence that inflation is moving

### Step 3: Remove Noise in FOMC Statements

In [88]:
def remove_noise(text):

    text = text.lower()

    tokens = word_tokenize(text)

    tokens = [word for word in tokens if word.isalpha() and word not in stopwords.words('english')]

    cleaned_text = ' '.join(tokens)

    return cleaned_text

fomc_statements['cleaned_statement'] = fomc_statements['statement'].apply(remove_noise)


In [89]:
fomc_statements.to_pickle('data/fomc_statements.pkl')